In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('csv/wisconsin_breast_cancer.csv')
df.drop(columns=['id','Unnamed: 32'],inplace=True)
df['diagnosis'].replace(['M','B'],[0,1],inplace=True)
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,0,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,0,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,0,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [3]:
y=df['diagnosis']
y=np.array(y).reshape(-1,1)
x=df.iloc[:,1:]
print(y.shape,x.shape)

(569, 1) (569, 30)


In [4]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [10]:
class logistic:
    def __init__(self,theta0_initial=None,epsilon=0.001,tol=0.001,max_iter=3000,theta_initial=None):
        self.theta_initial=theta_initial
        self.theta0_initial=theta0_initial
        self.epsilon=epsilon
        self.tol=tol
        self.max_iter=max_iter
        
    def fit(self,X,Y):
        if X.shape[0]!=Y.shape[0]:
            raise Exception('Dimensions of x and y are not same')
        self.theta_initial=np.random.normal(0,1,size=(X.shape[1],1))
        self.theta0_initial=0
        counter=0
        
        def sigmoid(theta0,theta):
            return 1/(1+np.exp(-(theta0+np.matmul(x,theta))))

        def del_t_by_del_theta0(theta0,theta):
            return np.sum(sigmoid(theta0,theta)-y)

        def del_t_by_del_theta(theta0,theta):
            return np.matmul((sigmoid(theta0,theta)-y).T,x).T

        def binary_cross_entropy(theta0,theta):
            return -(np.dot(y.T,np.log(sigmoid(theta0,theta)))+np.dot((1-y).T,np.log(1-sigmoid(theta0,theta))))
        
        while counter<=self.max_iter:
            theta0_final=self.theta0_initial-self.epsilon*del_t_by_del_theta0(self.theta0_initial,self.theta_initial)
            theta_final=self.theta_initial-self.epsilon*del_t_by_del_theta(self.theta0_initial,self.theta_initial)

            if np.abs(binary_cross_entropy(theta0_final,theta_final) - binary_cross_entropy(self.theta0_initial,self.theta_initial))<self.tol:
                break

            print(f'The value of binary cross entropy at epoch # {counter} is {binary_cross_entropy(theta0_final,theta_final)}')
            self.theta0_initial=theta0_final
            self.theta_initial=theta_final
            counter+=1
                
    def predict(self,test_x):
        def sigmoid(theta0,theta):
            return 1/(1+np.exp(-(theta0+np.matmul(test_x,theta))))
        predicted_value=sigmoid(self.theta0_initial,self.theta_initial)
        pred_class=[1 if i>0.5 else 0 for i in predicted_value]

        return np.array(pred_class)
    
    def accuracy(self,pred_y,test_y):
        if len(pred_y)==len(test_y):
            return np.sum(pred_y==test_y)/len(pred_y)
        else:
            raise Exception('Length of test_y and pred_y must be same')

In [13]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2)

In [19]:
epsilon=[0.0001,0.001,0.01,0.1]
tol=[0.00001,0.0001,0.001,0.01,0.1]
max_iter=[10,50,100,200,500]

In [20]:
ans=[]
accuracy=0
for i in epsilon:
    for j in tol:
        for k in max_iter:
            lr=logistic(max_iter=k,epsilon=i,tol=j)
            lr.fit(train_x,train_y)
            y_pred=lr.predict(test_x)
            if accuracy<lr.accuracy(y_pred,test_y):
                accuracy=lr.accuracy(y_pred,test_y)
                ans=[i,j,k]

The value of binary cross entropy at epoch # 0 is [[1802.21785551]]
The value of binary cross entropy at epoch # 1 is [[1621.045631]]
The value of binary cross entropy at epoch # 2 is [[1453.03453699]]
The value of binary cross entropy at epoch # 3 is [[1299.7756348]]
The value of binary cross entropy at epoch # 4 is [[1162.87228187]]
The value of binary cross entropy at epoch # 5 is [[1043.12421538]]
The value of binary cross entropy at epoch # 6 is [[940.02582052]]
The value of binary cross entropy at epoch # 7 is [[852.29283748]]
The value of binary cross entropy at epoch # 8 is [[778.32337309]]
The value of binary cross entropy at epoch # 9 is [[716.28179049]]
The value of binary cross entropy at epoch # 10 is [[664.31256766]]
The value of binary cross entropy at epoch # 0 is [[196.39361643]]
The value of binary cross entropy at epoch # 1 is [[193.57330602]]
The value of binary cross entropy at epoch # 2 is [[190.80946749]]
The value of binary cross entropy at epoch # 3 is [[188.10

The value of binary cross entropy at epoch # 87 is [[127.18873012]]
The value of binary cross entropy at epoch # 88 is [[125.97962378]]
The value of binary cross entropy at epoch # 89 is [[124.79471843]]
The value of binary cross entropy at epoch # 90 is [[123.63328043]]
The value of binary cross entropy at epoch # 91 is [[122.49460739]]
The value of binary cross entropy at epoch # 92 is [[121.37802659]]
The value of binary cross entropy at epoch # 93 is [[120.28289343]]
The value of binary cross entropy at epoch # 94 is [[119.20859013]]
The value of binary cross entropy at epoch # 95 is [[118.15452436]]
The value of binary cross entropy at epoch # 96 is [[117.12012802]]
The value of binary cross entropy at epoch # 97 is [[116.10485606]]
The value of binary cross entropy at epoch # 98 is [[115.1081854]]
The value of binary cross entropy at epoch # 99 is [[114.12961384]]
The value of binary cross entropy at epoch # 100 is [[113.16865908]]
The value of binary cross entropy at epoch # 0 i

The value of binary cross entropy at epoch # 148 is [[84.21299556]]
The value of binary cross entropy at epoch # 149 is [[84.0013744]]
The value of binary cross entropy at epoch # 150 is [[83.79221774]]
The value of binary cross entropy at epoch # 151 is [[83.58547946]]
The value of binary cross entropy at epoch # 152 is [[83.38111463]]
The value of binary cross entropy at epoch # 153 is [[83.17907946]]
The value of binary cross entropy at epoch # 154 is [[82.97933127]]
The value of binary cross entropy at epoch # 155 is [[82.7818284]]
The value of binary cross entropy at epoch # 156 is [[82.58653027]]
The value of binary cross entropy at epoch # 157 is [[82.39339726]]
The value of binary cross entropy at epoch # 158 is [[82.20239073]]
The value of binary cross entropy at epoch # 159 is [[82.01347297]]
The value of binary cross entropy at epoch # 160 is [[81.82660717]]
The value of binary cross entropy at epoch # 161 is [[81.64175741]]
The value of binary cross entropy at epoch # 162 i

The value of binary cross entropy at epoch # 123 is [[137.54395304]]
The value of binary cross entropy at epoch # 124 is [[136.22877656]]
The value of binary cross entropy at epoch # 125 is [[134.93620005]]
The value of binary cross entropy at epoch # 126 is [[133.66613626]]
The value of binary cross entropy at epoch # 127 is [[132.41848054]]
The value of binary cross entropy at epoch # 128 is [[131.1931095]]
The value of binary cross entropy at epoch # 129 is [[129.98988019]]
The value of binary cross entropy at epoch # 130 is [[128.80862949]]
The value of binary cross entropy at epoch # 131 is [[127.64917391]]
The value of binary cross entropy at epoch # 132 is [[126.51130967]]
The value of binary cross entropy at epoch # 133 is [[125.39481312]]
The value of binary cross entropy at epoch # 134 is [[124.29944144]]
The value of binary cross entropy at epoch # 135 is [[123.22493351]]
The value of binary cross entropy at epoch # 136 is [[122.17101107]]
The value of binary cross entropy a

The value of binary cross entropy at epoch # 296 is [[59.66106791]]
The value of binary cross entropy at epoch # 297 is [[59.50761932]]
The value of binary cross entropy at epoch # 298 is [[59.35547268]]
The value of binary cross entropy at epoch # 299 is [[59.20462043]]
The value of binary cross entropy at epoch # 300 is [[59.05505508]]
The value of binary cross entropy at epoch # 301 is [[58.90676913]]
The value of binary cross entropy at epoch # 302 is [[58.75975512]]
The value of binary cross entropy at epoch # 303 is [[58.61400559]]
The value of binary cross entropy at epoch # 304 is [[58.46951307]]
The value of binary cross entropy at epoch # 305 is [[58.32627007]]
The value of binary cross entropy at epoch # 306 is [[58.18426907]]
The value of binary cross entropy at epoch # 307 is [[58.04350254]]
The value of binary cross entropy at epoch # 308 is [[57.90396289]]
The value of binary cross entropy at epoch # 309 is [[57.76564246]]
The value of binary cross entropy at epoch # 310

...

The value of binary cross entropy at epoch # 100 is [[nan]]
The value of binary cross entropy at epoch # 0 is [[nan]]
The value of binary cross entropy at epoch # 1 is [[nan]]
The value of binary cross entropy at epoch # 2 is [[nan]]
The value of binary cross entropy at epoch # 3 is [[nan]]
The value of binary cross entropy at epoch # 4 is [[nan]]
The value of binary cross entropy at epoch # 5 is [[nan]]
The value of binary cross entropy at epoch # 6 is [[nan]]
The value of binary cross entropy at epoch # 7 is [[nan]]
The value of binary cross entropy at epoch # 8 is [[nan]]
The value of binary cross entropy at epoch # 9 is [[nan]]
The value of binary cross entropy at epoch # 10 is [[nan]]
The value of binary cross entropy at epoch # 11 is [[nan]]
The value of binary cross entropy at epoch # 12 is [[nan]]
The value of binary cross entropy at epoch # 13 is [[nan]]
The value of binary cross entropy at epoch # 14 is [[nan]]
The value of binary cross entropy at epoch # 15 is [[nan]]
The va

The value of binary cross entropy at epoch # 146 is [[nan]]
The value of binary cross entropy at epoch # 147 is [[nan]]
The value of binary cross entropy at epoch # 148 is [[nan]]
The value of binary cross entropy at epoch # 149 is [[nan]]
The value of binary cross entropy at epoch # 150 is [[nan]]
The value of binary cross entropy at epoch # 151 is [[nan]]
The value of binary cross entropy at epoch # 152 is [[nan]]
The value of binary cross entropy at epoch # 153 is [[nan]]
The value of binary cross entropy at epoch # 154 is [[nan]]
The value of binary cross entropy at epoch # 155 is [[nan]]
The value of binary cross entropy at epoch # 156 is [[nan]]
The value of binary cross entropy at epoch # 157 is [[nan]]
The value of binary cross entropy at epoch # 158 is [[nan]]
The value of binary cross entropy at epoch # 159 is [[nan]]
The value of binary cross entropy at epoch # 160 is [[nan]]
The value of binary cross entropy at epoch # 161 is [[nan]]
The value of binary cross entropy at epo

The value of binary cross entropy at epoch # 91 is [[nan]]
The value of binary cross entropy at epoch # 92 is [[nan]]
The value of binary cross entropy at epoch # 93 is [[nan]]
The value of binary cross entropy at epoch # 94 is [[nan]]
The value of binary cross entropy at epoch # 95 is [[nan]]
The value of binary cross entropy at epoch # 96 is [[nan]]
The value of binary cross entropy at epoch # 97 is [[nan]]
The value of binary cross entropy at epoch # 98 is [[nan]]
The value of binary cross entropy at epoch # 99 is [[nan]]
The value of binary cross entropy at epoch # 100 is [[nan]]
The value of binary cross entropy at epoch # 101 is [[nan]]
The value of binary cross entropy at epoch # 102 is [[nan]]
The value of binary cross entropy at epoch # 103 is [[nan]]
The value of binary cross entropy at epoch # 104 is [[nan]]
The value of binary cross entropy at epoch # 105 is [[nan]]
The value of binary cross entropy at epoch # 106 is [[nan]]
The value of binary cross entropy at epoch # 107 

The value of binary cross entropy at epoch # 243 is [[nan]]
The value of binary cross entropy at epoch # 244 is [[nan]]
The value of binary cross entropy at epoch # 245 is [[nan]]
The value of binary cross entropy at epoch # 246 is [[nan]]
The value of binary cross entropy at epoch # 247 is [[nan]]
The value of binary cross entropy at epoch # 248 is [[nan]]
The value of binary cross entropy at epoch # 249 is [[nan]]
The value of binary cross entropy at epoch # 250 is [[nan]]
The value of binary cross entropy at epoch # 251 is [[nan]]
The value of binary cross entropy at epoch # 252 is [[nan]]
The value of binary cross entropy at epoch # 253 is [[nan]]
The value of binary cross entropy at epoch # 254 is [[nan]]
The value of binary cross entropy at epoch # 255 is [[nan]]
The value of binary cross entropy at epoch # 256 is [[nan]]
The value of binary cross entropy at epoch # 257 is [[nan]]
The value of binary cross entropy at epoch # 258 is [[nan]]
The value of binary cross entropy at epo

The value of binary cross entropy at epoch # 400 is [[nan]]
The value of binary cross entropy at epoch # 401 is [[nan]]
The value of binary cross entropy at epoch # 402 is [[nan]]
The value of binary cross entropy at epoch # 403 is [[nan]]
The value of binary cross entropy at epoch # 404 is [[nan]]
The value of binary cross entropy at epoch # 405 is [[nan]]
The value of binary cross entropy at epoch # 406 is [[nan]]
The value of binary cross entropy at epoch # 407 is [[nan]]
The value of binary cross entropy at epoch # 408 is [[nan]]
The value of binary cross entropy at epoch # 409 is [[nan]]
The value of binary cross entropy at epoch # 410 is [[nan]]
The value of binary cross entropy at epoch # 411 is [[nan]]
The value of binary cross entropy at epoch # 412 is [[nan]]
The value of binary cross entropy at epoch # 413 is [[nan]]
The value of binary cross entropy at epoch # 414 is [[nan]]
The value of binary cross entropy at epoch # 415 is [[nan]]
The value of binary cross entropy at epo

In [21]:
print('The best accuracy ',accuracy)

The best accuracy  63.31578947368421


In [22]:
print(f'epsilon : {ans[0]}       tol : {ans[1]}           max_iter : {ans[2]}')

epsilon : 0.0001       tol : 1e-05           max_iter : 500
